In [ ]:
!pip install transformers datasets seqeval tokenizers -q
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/chriswu99aaa/MeTNet.git

Cloning into 'MeTNet'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 241 (delta 6), reused 0 (delta 0), pack-reused 228
Receiving objects: 100% (241/241), 32.24 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (128/128), done.


First to use the method provided in the library to load the file

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from pathlib import Path
import re

def read_file(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
texts, tags = read_file('/content/MeTNet/data/Few-COMM/train.txt')


In [ ]:
print(texts[0][:], tags[0][:], sep='\n')

['(', '8', '7', ')', '吊', '带', '蕾', '丝', '睡', '衣', '女', '夏', '季', '短', '袖', '短', '裤', '两', '件', '套', '装', '性', '感', '夏', '天', '冰', '丝', '家', '居', '服', '(', '带', '胸', '垫', ')', '可', '外', '穿']
['O', 'O', 'O', 'O', 'O', 'O', 'B-材质', 'I-材质', 'O', 'O', 'O', 'B-适用季节', 'I-适用季节', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-其他属性', 'I-其他属性', 'I-其他属性', 'I-其他属性', 'B-材质', 'I-材质', 'O', 'O', 'O', 'O', 'B-版型', 'I-版型', 'I-版型', 'O', 'B-功能功效', 'I-功能功效', 'I-功能功效']


In [ ]:
tags

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
tag2id

{'B-材质': 0,
 'B-酸碱度': 1,
 'I-控制方式': 2,
 'I-是否净洗': 3,
 'B-筒高': 4,
 'I-配件类型': 5,
 'B-长短': 6,
 'I-长短': 7,
 'B-鞋垫材质': 8,
 'I-糖含量': 9,
 'I-厚薄': 10,
 'I-长度': 11,
 'I-填充材质': 12,
 'B-长度': 13,
 'B-是否有机': 14,
 'I-颜色': 15,
 'I-适用运营商': 16,
 'B-填充材质': 17,
 'B-控制方式': 18,
 'B-香型': 19,
 'I-果肉颜色': 20,
 'B-商品特色': 21,
 'B-分类': 22,
 'B-厚度': 23,
 'I-其他属性': 24,
 'B-大小': 25,
 'B-成分': 26,
 'I-版型': 27,
 'I-产地': 28,
 'B-适用衣物': 29,
 'B-适用运营商': 30,
 'I-材质': 31,
 'I-厚度': 32,
 'I-存储容量': 33,
 'B-送礼对象': 34,
 'B-糖含量': 35,
 'B-礼盒类型': 36,
 'I-礼盒类型': 37,
 'B-袖型': 38,
 'I-袖长': 39,
 'I-供电方式': 40,
 'B-品质等级': 41,
 'B-功能功效': 42,
 'I-是否去皮': 43,
 'I-筒高': 44,
 'B-袖长': 45,
 'I-商品特色': 46,
 'I-领型': 47,
 'B-是否净洗': 48,
 'I-国产/进口': 49,
 'B-脂肪含量': 50,
 'B-适用对象': 51,
 'I-运输服务': 52,
 'B-造型': 53,
 'B-其他属性': 54,
 'B-产地': 55,
 'B-锅底类型': 56,
 'B-组合形式': 57,
 'I-造型': 58,
 'B-配件类型': 59,
 'B-版型': 60,
 'I-接口': 61,
 'B-是否去骨': 62,
 'I-香型': 63,
 'I-锅底类型': 64,
 'I-功能功效': 65,
 'B-领型': 66,
 'B-面料材质': 67,
 'I-适用空间': 68,
 'B-定制服务': 69,
 'I-成分': 70,
 'B-运

In [ ]:
len(unique_tags)

111

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
import torch

class FewCommDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")


train_dataset = FewCommDataset(train_encodings, train_labels)
val_dataset = FewCommDataset(val_encodings, val_labels)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

## Evaluate



In [ ]:
import evaluate

seqeval = evaluate.load('seqeval')

In [ ]:
label_list = []
for i in range(len(id2tag)):
    label_list.append(id2tag[i])

In [ ]:
label_list

['B-国产/进口',
 'B-适用人群',
 'I-适用人群',
 'B-适用空间',
 'I-版型',
 'B-面料材质',
 'B-产地',
 'B-酸碱度',
 'B-果肉颜色',
 'B-锅底类型',
 'B-是否去骨',
 'I-适用空间',
 'I-是否有机',
 'B-外观',
 'B-袖长',
 'I-长短',
 'B-组合形式',
 'I-送礼对象',
 'I-袖型',
 'I-型号',
 'B-型号',
 'B-形状形态',
 'I-接口',
 'I-是否净洗',
 'I-供电方式',
 'B-送礼对象',
 'B-是否去皮',
 'B-是否净洗',
 'I-适用人数',
 'I-定制服务',
 'B-定制服务',
 'I-适用对象',
 'B-供电方式',
 'I-产地',
 'I-配件类型',
 'I-脂肪含量',
 'I-大小',
 'B-其他属性',
 'O',
 'B-厚薄',
 'I-糖含量',
 'I-面料材质',
 'I-袖长',
 'I-造型',
 'I-颜色',
 'I-是否去骨',
 'B-赠品',
 'I-其他属性',
 'I-品质等级',
 'I-适用运营商',
 'I-组合形式',
 'I-果肉颜色',
 'B-接口',
 'I-功能功效',
 'B-适用人数',
 'B-版型',
 'B-材质',
 'I-厚度',
 'B-功能功效',
 'B-长短',
 'B-造型',
 'I-成分',
 'I-填充材质',
 'I-是否去皮',
 'B-适用衣物',
 'I-国产/进口',
 'I-鞋垫材质',
 'B-填充材质',
 'B-运输服务',
 'I-控制方式',
 'I-适用衣物',
 'B-品质等级',
 'B-适用运营商',
 'B-适用对象',
 'I-存储容量',
 'I-香型',
 'I-运输服务',
 'I-商品特色',
 'B-脂肪含量',
 'I-厚薄',
 'B-控制方式',
 'B-筒高',
 'I-长度',
 'I-外观',
 'B-领型',
 'B-分类',
 'I-适用季节',
 'B-适用季节',
 'B-糖含量',
 'B-存储容量',
 'I-分类',
 'B-商品特色',
 'I-赠品',
 'B-香型',
 'B-是否有机',
 'I-材质',
 'I-酸碱度',
 'B-厚度

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p,l) in zip(prediction, label) if l!=-100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p,l) in zip(prediction, label) if l!= -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references = true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Training

In [ ]:
from transformers import BertForTokenClassification, AdamW, Trainer, TrainingArguments
from torch.utils.data import DataLoader

from datasets import load_metric

model = BertForTokenClassification.from_pretrained('bert-base-chinese', num_labels=len(unique_tags), id2label=id2tag, label2id=tag2id)


training_args = TrainingArguments(
    output_dir='bert_ner_model',     # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    push_to_hub=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,

)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset (only 5 in few shot)
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

trainer.train()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.337600,0.270429,0.732635,0.742468,0.737519,0.911288
2,0.198600,0.238871,0.767646,0.789944,0.778636,0.922556


TrainOutput(global_step=5078, training_loss=0.43349526042588915, metrics={'train_runtime': 927.9579, 'train_samples_per_second': 87.528, 'train_steps_per_second': 5.472, 'total_flos': 2074605438209400.0, 'train_loss': 0.43349526042588915, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/chriswu88/bert_ner_model/commit/cdb4c04ab66a30aadb3715c992cfe6554eb64692', commit_message='End of training', commit_description='', oid='cdb4c04ab66a30aadb3715c992cfe6554eb64692', pr_url=None, pr_revision=None, pr_num=None)

## Inference


In [ ]:
text = "夏日冰凉西瓜。"